## Economic Evaluation of the Models

In this notebook, we employ volatility forecasts to construct portfolios and evaluate their economic value. We use `scipy.optimize` to find the optimal weights for the portfolio.

Besides the minimum volatility portfolio, we also consider a maximum Sharpe ratio portfolio, which is constructed by maximizing the Sharpe ratio.

In our empirical analysis, we constrained the portfolio weight to be in [0, 1] (producing a long-only portfolio), but this is not crucial for our results. To be more comprehensive, we can follow Bollerslev et al. (2019) and add a short constraint.

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

class portfolio():
    def __init__(self, model, forecast_horizon, short, portfolio_type='min_volatility'):
        if model == 'equal':
            forecast_log_rv = pd.read_csv(f'../results/forecasts/GRU_h_{forecast_horizon}.csv', index_col=0)
        elif model == 'optimal':
            date = pd.read_csv(f'../results/forecasts/GRU_h_{forecast_horizon}.csv', index_col=0).index
            # use true log rv
            forecast_log_rv = pd.read_csv('../data/realized_volatility/log_rv_table.csv', index_col=0).loc[date, :]
        else:
            forecast_log_rv = pd.read_csv(f'../results/forecasts/{model}_h_{forecast_horizon}.csv', index_col=0)

        true_log_rv = pd.read_csv('../data/realized_volatility/log_rv_table.csv', index_col=0).loc[forecast_log_rv.index, :]
        self.true_return = pd.read_csv('../data/return_table.csv', index_col=0).loc[:, forecast_log_rv.columns]

        self.forecast_horizon = forecast_horizon
        self.update_freq = 1
        self.short = short
        self.stock_num = forecast_log_rv.shape[1]
        self.total_date = list(forecast_log_rv.index)
        self.model = model
        self.portfolio_type = portfolio_type
        
        stock_list = pd.Series(forecast_log_rv.columns)
        self.stock_list = list(stock_list)
        self.stock_num = len(stock_list)

        self.forecast = np.exp(forecast_log_rv.loc[:, stock_list])
        self.true = np.exp(true_log_rv.loc[:, stock_list])
        self.true_return = self.true_return.loc[:, stock_list]

    def ex_post_sharpe(self):
        return_list = []
        for i in range(len(self.trading_day)):
            return_day = self.true_return.loc[self.trading_day[i], :]
            weight = self.weights[i, :]
            return_list.append(np.sum(weight * return_day))
        volatility_list = []
        for i in range(len(self.trading_day)):
            weight = self.weights[i, :]
            ture_rv_day = self.true.loc[self.trading_day[i], :]
            volatility_list.append(np.sum(np.square(weight) * ture_rv_day))
        return np.mean(np.array(return_list) / np.sqrt(np.array(volatility_list)))

    def turnover(self):
        turnover_list = []
        for i in range(1, len(self.trading_day)):
            weight_prev = np.array(list(self.weights[i - 1, :]))
            weight_day = np.array(list(self.weights[i, :]))
            return_day = np.array(list(self.true_return.loc[self.trading_day[i], :]))
            summation = 0
            for j in range(self.stock_num):
                summation += np.abs(weight_day[j] - (weight_prev[j] * (1 + return_day[j]) / (1 + np.sum(weight_prev * return_day))))
            turnover_list.append(summation)
        return np.mean(turnover_list)

    def ex_post_variation(self):
        variation_list = []
        for i in range(len(self.trading_day)):
            true_day = np.array(list(self.true.loc[self.trading_day[i], :]))
            weight = np.array(list(self.weights[i, :]))
            variation = np.sum(np.square(weight) * true_day)
            variation_list.append(variation)
        return np.mean(variation_list)
    
    def get_trading_day(self):
        self.trading_day = []
        day = 0
        while day + self.update_freq <= len(self.total_date):
            self.trading_day.append(self.total_date[day])
            day += self.update_freq
        
        self.weights = np.zeros((len(self.trading_day), self.stock_num))
    
    def construction(self):
        self.get_trading_day()
        
        for i, day in enumerate(self.trading_day):
            forecast_day = self.forecast.loc[day, :]
            true_return_day = self.true_return.loc[day, :]
            optimal_weight = self.optimization(forecast_day, true_return_day)
            self.weights[i, :] = optimal_weight
        
        if self.portfolio_type == 'min_volatility':
            return self.ex_post_variation(), -1, self.turnover()
        elif self.portfolio_type == 'max_sharpe':
            return -1, self.ex_post_sharpe(), self.turnover()
    
    def optimization(self, forecast_day, true_return_day):
        def obj_max_sharpe(weight, forecast_day, true_return_day):
            return -(np.sum(true_return_day * weight) / np.sqrt(np.sum(forecast_day * np.square(weight))))
        def obj_min_volatility(weight, forecast_day):
            return np.sum(forecast_day * np.square(weight))
        
        def constraint_1(weight):
            return np.sum(weight) - 1
        def constraint_2(weight):
            return 1 + 2 * self.short - np.sum(np.abs(weight))
        constraint_sum = {'type' : 'eq', 'fun' : constraint_1}
        constraint_short = {'type' : 'ineq', 'fun' : constraint_2}
        
        init = [1 / self.stock_num for i in range(self.stock_num)]
        if self.model == 'equal':
            return init
        
        # short == -1 : allow infinite short
        # short == -2 : long-only
        # otherwise : different short exposure level
        if self.short == -1:
            bounds_lim = [(-1, 1) for i in range(self.stock_num)]
            constraints = [constraint_sum]
        elif self.short == -2:
            bounds_lim = [(0, 1) for i in range(self.stock_num)]
            constraints = [constraint_sum]
        else:
            bounds_lim = [(-1, 1) for i in range(self.stock_num)]
            constraints = [constraint_sum, constraint_short]
        
        if self.portfolio_type == 'max_sharpe':
            optimal_weight = minimize(fun=obj_max_sharpe, args=(forecast_day, true_return_day), x0=init, constraints=constraints, bounds=bounds_lim)
        elif self.portfolio_type == 'min_volatility':
            optimal_weight = minimize(fun=obj_min_volatility, args=(forecast_day), x0=init, constraints=constraints, bounds=bounds_lim)
        
        return list(optimal_weight.x)

### Long-Only Portfolio

The long-only minimum volatility portfolio is constructed by minimizing the portfolio variance. This optimization is subject to two constraints: the sum of the weights must equal 1, and each weight must lie within the range [0, 1].

In [4]:
def min_vol():
    model_list = ['optimal', 'equal', 'VAR', 'LSTM', 'GRU', 'MLSTMF', 'MGRUF']
    horizons = [1, 5, 22]
    # s = -2 means long only
    df = pd.DataFrame(index=model_list, columns=horizons)
    for h in horizons:
        for model in model_list:
            volatility, _, _ = portfolio(model, h, -2, 'min_volatility').construction()
            df.loc[model, h] = volatility
    df = df.div(df.loc['equal'], axis=1)
    # df.to_csv('../results/evaluations/min_volatility_longonly.csv')
    return df

pd.options.display.float_format = '{:.3f}'.format
min_vol()

,1,5,22
optimal,0.602,0.602,0.600
equal,1.000,1.000,1.000
VAR,0.807,0.882,0.967
LSTM,0.852,0.890,0.922
GRU,0.835,0.883,0.924
MLSTMF,0.837,0.889,0.919
MGRUF,0.826,0.873,0.911


### Portfolio with Short Selling

The long-short portfolio is constructed by either minimizing the portfolio variance or maximizing the Sharpe ratio, subject to two constraints: the sum of the weights must equal 1, and a short constraint is applied (for more details, please refer to Bollerslev et al. (2019)).

In [5]:
def portfolio_with_short(portfolio_type):
    model_list = ['optimal', 'equal', 'VAR', 'LSTM', 'GRU', 'MLSTMF', 'MGRUF']
    h = 1
    short_list = [0, 0.1, 0.2, 0.5, 1]
    
    df = pd.DataFrame(index=model_list, columns=short_list)
    for model in model_list:
        for short in short_list:
            volatility, sharpe, _ = portfolio(model, h, short, portfolio_type).construction()
            if portfolio_type == 'min_volatility':
                df.loc[model, short] = volatility
            elif portfolio_type == 'max_sharpe':
                df.loc[model, short] = sharpe
    if portfolio_type == 'min_volatility':
        df = df.div(df.loc['equal'], axis=1)
    # df.to_csv(f'../results/evaluations/{portfolio_type}_longshort_h_{h}.csv')
    return df

It is observed that regardless of whether $s = 0$ (long only) or $s > 0$ (long and short), the memory-augmented model outperforms the original model in terms of out-of-sample risk forecasting.

In [6]:
pd.options.display.float_format = '{:.5f}'.format
portfolio_with_short('min_volatility')

,0.00000,0.10000,0.20000,0.50000,1.00000
optimal,0.63145,0.60208,0.60208,0.60208,0.60208
equal,1.00000,1.00000,1.00000,1.00000,1.00000
VAR,0.81777,0.80690,0.80690,0.80690,0.80690
LSTM,0.85900,0.85148,0.85148,0.85148,0.85148
GRU,0.84771,0.83502,0.83502,0.83502,0.83502
MLSTMF,0.85233,0.83669,0.83669,0.83669,0.83669
MGRUF,0.83931,0.82586,0.82586,0.82586,0.82586


In [7]:
pd.options.display.float_format = '{:.3f}'.format
portfolio_with_short('max_sharpe')

,0.000,0.100,0.200,0.500,1.000
optimal,6.476,12.896,12.919,12.987,13.102
equal,-0.453,-0.453,-0.453,-0.453,-0.453
VAR,5.307,10.189,10.211,10.268,10.368
LSTM,5.332,9.849,9.875,9.942,10.069
GRU,5.426,9.982,10.005,10.073,10.188
MLSTMF,5.336,9.948,9.967,10.036,10.153
MGRUF,5.291,10.019,10.039,10.111,10.226
